In [8]:
import openai
import os
import json
OPENAI_KEY = str(json.load(open("openai_key.json"))['key'])
openai.api_key = OPENAI_KEY
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from generate_robot_instruction import encode_task_generation_prompt, generate_task_data, encode_instruct_prompt, generate_instruction_following_chat_data, post_process_chat_response

### Generate Tasks

In [10]:
generate_task_data(
    output_dir="./gpt4_generation/task_generation_v2/",
    num_tasks_to_generate=200,
    prompt_file_name="./prompts_for_gpt/robot_task_prompt_v2.txt",
    model_name="gpt-4-0314"
)

  0%|          | 0/200 [00:00<?, ?it/s]

### [[{'role': 'user', 'content': 'You are serving as a task-generation helper for a given robot environment.\xa0\n\n[General Environment Description] \nThe environment is called "Block World". \nThere is a 7DOF Franka Panda robot with a parallel gripper, it has a force sensor on the end effector. The robot is mounted on a table.\nThere are multiple blocks on the table, here we use 2 blocks as an example. The blocks are called {block A} and {block B}. \nThe blocks are initialized at a random position on the table.\nThe observation space (when there are two blocks) is a 26 dimension vector, consisting of:\nblock A position(3), block A orientation(4), block B position(3), block B orientation(4), end effector position(3), end effector orientation(4), end effector force(3), left finger position(1), right finger position(1) \nIf there are more than two blocks, the observation space is a 26+7*(n-2) dimension vector, where n is the number of blocks.\n\n[Extra Environment Assumption Tags]\n<we



prompts: 100%|██████████| 1/1 [01:34<00:00, 94.20s/it]


['', ' Task 1: <weight> Pick up the heavier block\n', ' Task 2: <move> Find the movable block and place it on top of the other block*\n', ' Task 3: <weight> Move the heavier block to the corner of the table*\n', ' Task 4: <three><weight> Sort all the blocks by their weight*\n', ' Task 5: <three> Stack the three blocks\n', ' Task 6: <weight> Pick up the lighter block\n', ' Task 7: <three> Pick up the middle block in height\n', ' Task 8: <four> Move the heaviest block to the left corner of the table, and the lightest block to the right corner*\n', ' Task 9: <four> Move the two blocks in the middle to the back edge of the table*\n', ' Task 10: <move> Pick up the only movable block and hold it for 5 seconds*\n', ' Task 11: <weight><move> Pick up the movable and lighter block\n', ' Task 12: <four> Stack two of the blocks together\n', ' Task 13: <four> Arrange the blocks in a square formation*\n', ' Task 14: <three> Place the blocks in a straight line with equal distance between them*\n', ' 



prompts: 100%|██████████| 1/1 [01:46<00:00, 106.44s/it]


['', ' Task 1: <weight> Pick up the heaviest block*\n', ' Task 2: <move> Find the movable block and place it on top of the other block*\n', ' Task 3: <weight> Move the heavier block to the corner of the table*\n', ' Task 4: <three><weight> Sort all the blocks by their weight*\n', ' Task 5: <three> Stack the three blocks*\n\n', ' Task 6: <four> Stack the four blocks in a 2x2 square*\n', ' Task 7: <move><three> Move the immovable block between the two movable blocks.\n', ' Task 8: <weight><three> Place the lightest block on top of the heaviest block*\n', ' Task 9: <four> Create a pyramid with the four blocks*\n', ' Task 10: <move> Move a movable block to the edge of the table*\n\n', ' Task 11: <weight><three> Move the lightest block on top of the medium-weight block*\n', ' Task 12: <move><four> Move all the movable blocks to one corner of the table*\n', ' Task 13: <weight> Move the heaviest block on top of the lightest block*\n', ' Task 14: <three> Create a straight line with the three b



prompts: 100%|██████████| 1/1 [01:40<00:00, 100.24s/it]


['', ' Task 1: <weight> Pick up the heavier block and place it on top of the lighter block*\n', ' Task 2: <move> Find the movable block and place it on top of the other block.\n', ' Task 3: <weight> Move the heavier block to the corner of the table.\n', ' Task 4: <three><weight> Sort all the blocks by their weight.\n', ' Task 5: <three> Stack the three blocks.\n', ' Task 6: <weight> Pick up the lighter block and place it on the other side of the table.\n', ' Task 7: <move> Stack the movable block on top of the unmovable block.\n', ' Task 8: <three> Create a bridge between the three blocks horizontally.\n', ' Task 9: <four> Make a pyramid using all four blocks.\n', ' Task 10: <weight> Elevate the lighter block as high as possible.\n', ' Task 11: <move> Transfer the movable block to a designated location on the table.\n', ' Task 12: <weight> Set the two blocks next to each other in increasing weight order from left to right.\n', ' Task 13: <three> Arrange the three blocks in a row parall



100%|██████████| 200/200 [06:46<00:00,  2.03s/it]

['', ' Task 1: <weight> pick up the heavier block and place it on top of the lighter block*\n', ' Task 2: <move> find the movable block and place it on top of the other block*\n', ' Task 3: <weight> move the heavier block to the corner of the table\n', ' Task 4: <three><weight> sort all the blocks by their weight*\n', ' Task 5: <three> stack the three blocks\n\n', ' Task 6: <move> transfer the movable block to the opposite corner of the table\n', ' Task 7: <weight> place the lighter block on top of the heavier block, ensuring they are aligned*\n', ' Task 8: <three> create a triangle formation with the blocks on the table\n', ' Task 9: <three><move> find and stack the two movable blocks*\n', ' Task 10: <weight> swap the positions of the two blocks by weight*\n\n', ' Task 11: <three> arrange the blocks in a straight line, sorted by their positions on the table\n', ' Task 12: <four> create a 2x2 square formation with the four blocks\n', ' Task 13: <weight> balance the lighter block on top

In [11]:
import multiprocessing as mp

def f(x):
  return x * x

if __name__ == "__main__":
  with mp.Pool() as pool:
    result = pool.map(f, [1, 2, 3])
    print(result)

[1, 4, 9]


#### Split task data
Split it into tasks used by instruction generation.

In [12]:
import re, os
import string
import json
def split_json_file(json_file, 
                    output_path, n=2):
  """Splits a JSON file into N JSONL files, where each file contains n elements.

  Args:
    json_file: The path to the JSON file to split.
    n: The number of JSONL files to create.

  Returns:
    The list of paths to the created JSONL files.
  """

  with open(json_file, "r") as f:
    data = json.load(f)
  os.makedirs(output_path, exist_ok=True)
  jsonl_files = []
  for i in range(0, len(data), n):
    jsonl_file = f"{output_path}split_{i // n}.jsonl"
    with open(jsonl_file, "w") as outfile:
      for record in data[i : i + n]:
        json.dump(record, outfile)
        outfile.write("\n")

    jsonl_files.append(jsonl_file)
  return jsonl_files

In [13]:
jsonl_files = split_json_file(json_file="./gpt4_generation/task_generation_v2/task_regen.json",
                              output_path="./gpt4_generation/task_generation_v2/",
                              n=100)

### Generate COT instruct data

In [14]:
from generate_robot_instruction import generate_instruction_following_chat_data


ret = generate_instruction_following_chat_data(output_dir="./gpt4_generation/cot_full_v5",
                                        instruction_prompt_file="./prompts_for_gpt/robot_cot_prompts_v5.txt",
                                        seed_example_path="./prompts_for_gpt/seeded_example_cot_v4.jsonl",
                                        seed_tasks_path="./gpt4_generation/task_generation_v2/split_0.jsonl",
                                        num_instructions_to_generate=500,
                                        num_of_tasks_each_request=3,
                                        use_sub_skill_data=True,
                                        output_file="cot_instrut_regen.json"
                                        )

FileNotFoundError: [Errno 2] No such file or directory: './subtask_data/sub_skill_example_data.json'

In [36]:
ret = generate_instruction_following_chat_data(output_dir="./gpt4_generation/cot_full_v4",
                                        instruction_prompt_file="./prompts_for_gpt/robot_cot_prompts_v4.txt",
                                        seed_example_path="./prompts_for_gpt/seeded_example_cot_v4.jsonl",
                                        seed_tasks_path="./gpt4_generation/task_generation_v2/split_1.jsonl",
                                        num_instructions_to_generate=500,
                                        output_file="cot_instrut_regen_2.json"
                                        )

### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to simulate the task instructions and corresponding responses happening during task solving.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model performance on the generated responses.\n\n[General Environment Description] \nThe environment is called "Block World". \nThere is a 7DOF Franka Panda robot with a parallel gripper, it has a force sensor on the end effector. The robot is mounted on a table.\nThere are multiple blocks on tha table, here we use 2 blocks as example. The blocks are called {block A} and {block B}. If there are more than two blocks, they are called {block C}, {block D}, etc.\nThe blocks are initialized at a random position on the table.\nThe observation space (when there are two bl




prompts: 100%|██████████| 1/1 [02:32<00:00, 152.83s/it]




  1%|          | 5/500 [02:32<21:00:49, 152.83s/it]
Request 1 took 152.83s
Generated 5 instructions, kept 5 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to simulate the task instructions and corresponding responses happening during task solving.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model performance on the generated responses.\n\n[General Environment Description] \nThe environment is called "Block World". \nThere is a 7DOF Franka Panda robot with a parallel gripper, it has a force sensor on the end effector. The robot is mounted on a table.\nThere are multiple blocks on tha table, here we use 2 blocks as example. The blocks are called {block A} and {block B}. If there are more than two blocks, they are called {block C}, {bl







prompts: 100%|██████████| 1/1 [01:36<00:00, 96.17s/it]




  2%|▏         | 9/500 [04:08<4:51:39, 35.64s/it]
Request 2 took 96.17s
Generated 4 instructions, kept 4 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to simulate the task instructions and corresponding responses happening during task solving.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model performance on the generated responses.\n\n[General Environment Description] \nThe environment is called "Block World". \nThere is a 7DOF Franka Panda robot with a parallel gripper, it has a force sensor on the end effector. The robot is mounted on a table.\nThere are multiple blocks on tha table, here we use 2 blocks as example. The blocks are called {block A} and {block B}. If there are more than two blocks, they are called {block C}, {block

...

 98%|█████████▊| 492/500 [4:17:10<05:12, 39.09s/it]
Request 112 took 182.52s
Generated 7 instructions, kept 7 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to simulate the task instructions and corresponding responses happening during task solving.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model performance on the generated responses.\n\n[General Environment Description] \nThe environment is called "Block World". \nThere is a 7DOF Franka Panda robot with a parallel gripper, it has a force sensor on the end effector. The robot is mounted on a table.\nThere are multiple blocks on tha table, here we use 2 blocks as example. The blocks are called {block A} and {block B}. If there are more than two blocks, they are called {block C}, {







prompts: 100%|██████████| 1/1 [02:52<00:00, 172.12s/it]




100%|█████████▉| 498/500 [4:20:02<01:05, 32.67s/it]
Request 113 took 172.13s
Generated 6 instructions, kept 6 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to simulate the task instructions and corresponding responses happening during task solving.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model performance on the generated responses.\n\n[General Environment Description] \nThe environment is called "Block World". \nThere is a 7DOF Franka Panda robot with a parallel gripper, it has a force sensor on the end effector. The robot is mounted on a table.\nThere are multiple blocks on tha table, here we use 2 blocks as example. The blocks are called {block A} and {block B}. If there are more than two blocks, they are called {block C}, {







prompts: 100%|██████████| 1/1 [01:21<00:00, 81.11s/it]


100%|██████████| 500/500 [4:21:23<00:00, 31.37s/it]

100%|██████████| 500/500 [4:21:23<00:00, 25.93s/it]
Request 114 took 81.11s
Generated 2 instructions, kept 2 instructions


In [41]:
import utils
utils.full_inst_json_to_full_form_json('./gpt4_generation/cot_full_v4/cot_instrut_regen.json', 
                                       './gpt4_generation/cot_full_v4/cot_instrut_regen_formatted.json')

In [51]:
data_lists = list(open("./gpt4_generation_cot/cot_instrut_regen_gorilla.jsonl"))

In [52]:
data = data_lists[0]
data = json.loads(data)['code']
if '###Instruction.' in data:
    data = data.replace('###Instruction.', '###Instruction:')
if 'Output:' in data:
    a = {'instruction':data.split('Output:')[0].split('Instruction:')[1].split('###')[0],
        'input': data.split('Output:')[0].split('Input:')[1].split('###')[0],
        'output': data.split('Output:')[1]}

In [61]:
data.split('Output:')[1]

" {'explanation': No. We don't know the weight of the blocks, need to infer weight first., 'code': <nooutput>.}\n"